<a href="https://colab.research.google.com/github/prateekjoshi565/textrank_text_summarization/blob/master/TestRank_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jahan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# split the the text in the articles into sentences
# Read the CSV file
sentences = []
file = open("001-full-article.txt",'r', encoding='utf8')


content = [x.strip() for x in file.readlines()]
file.close()


    
for line in content:
    if line != '':
        sentences.append(sent_tokenize(line)) 

[['Claxton hunting first major medal'],
 ["British hurdler Sarah Claxton is confident she can win her first major medal at next month's European Indoor Championships in Madrid."],
 ['The 25-year-old has already smashed the British record over 60m hurdles twice this season, setting a new mark of 7.96 seconds to win the AAAs title.',
  '"I am quite confident," said Claxton.',
  '"But I take each race as it comes.',
  '"As long as I keep up my training but not do too much I think there is a chance of a medal."',
  'Claxton has won the national 60m hurdles title for the past three years but has struggled to translate her domestic success to the international stage.',
  'Now, the Scotland-born athlete owns the equal fifth-fastest time in the world this year.',
  "And at last week's Birmingham Grand Prix, Claxton left European medal favourite Russian Irina Shevchenko trailing in sixth spot."],
 ['For the first time, Claxton has only been preparing for a campaign over the hurdles - which coul

In [4]:
# flatten the list
sentences = [y for x in sentences for y in x]

In [5]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

['claxton hunting first major medal',
 'british hurdler sarah claxton is confident she can win her first major medal at next month s european indoor championships in madrid ',
 'the    year old has already smashed the british record over   m hurdles twice this season  setting a new mark of      seconds to win the aaas title ',
 ' i am quite confident   said claxton ',
 ' but i take each race as it comes ',
 ' as long as i keep up my training but not do too much i think there is a chance of a medal  ',
 'claxton has won the national   m hurdles title for the past three years but has struggled to translate her domestic success to the international stage ',
 'now  the scotland born athlete owns the equal fifth fastest time in the world this year ',
 'and at last week s birmingham grand prix  claxton left european medal favourite russian irina shevchenko trailing in sixth spot ',
 'for the first time  claxton has only been preparing for a campaign over the hurdles   which could explain her

In [6]:
nltk.download('stopwords')# one time execution

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jahan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [8]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [9]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [9]:
!python -m wget http://nlp.stanford.edu/data/glove.6B.zip


^C


In [10]:
!python -m unzip glove*.zip

C:\Users\jahan\anaconda3\python.exe: No module named unzip


In [10]:
# Extract word vectors
word_embeddings = {}
f = open("../model/glove.6B.50d.txt",'r', errors = 'ignore', encoding='utf8')


#f = open('../model/glove.6B.50d.txt', encoding='windows-1252')
for line in f:
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
    except:
        f.__next__()
    word_embeddings[word] = coefs
f.close()

StopIteration: 

In [11]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((50,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((50,))
    sentence_vectors.append(v)

In [12]:
len(sentence_vectors)

12

The next step is to find similarities among the sentences. We will use cosine similarity to find similarity between a pair of sentences. Let's create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [13]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,50), sentence_vectors[j].reshape(1,50))[0,0]

In [16]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [17]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [26]:
# Specify number of sentences to form the summary
sn = 5

final_sentences=''

for i in range(sn):
    final_sentences += ''.join(ranked_sentences[i][1])

In [27]:
#TODO : ROUGE to evaluate.
final_sentences
#ranked_sentences

"British hurdler Sarah Claxton is confident she can win her first major medal at next month's European Indoor Championships in Madrid.For the first time, Claxton has only been preparing for a campaign over the hurdles - which could explain her leap in form.In previous seasons, the 25-year-old also contested the long jump but since moving from Colchester to London she has re-focused her attentions.Now, the Scotland-born athlete owns the equal fifth-fastest time in the world this year.Claxton will see if her new training regime pays dividends at the European Indoors which take place on 5-6 March."

In [28]:

file = open("001-summary.txt",'r', encoding='utf8')

summary_sen=[]
summary = [x.strip() for x in file.readlines()]
file.close()
summary

['For the first time, Claxton has only been preparing for a campaign over the hurdles - which could explain her leap in form.Claxton has won the national 60m hurdles title for the past three years but has struggled to translate her domestic success to the international stage.British hurdler Sarah Claxton is confident she can win her first major medal at next month\'s European Indoor Championships in Madrid.Claxton will see if her new training regime pays dividends at the European Indoors which take place on 5-6 March."I am quite confident," said Claxton.']

In [36]:
actual_sentences = ''
actual_sentences += ''.join(summary[0])


In [37]:
actual_sentences

'For the first time, Claxton has only been preparing for a campaign over the hurdles - which could explain her leap in form.Claxton has won the national 60m hurdles title for the past three years but has struggled to translate her domestic success to the international stage.British hurdler Sarah Claxton is confident she can win her first major medal at next month\'s European Indoor Championships in Madrid.Claxton will see if her new training regime pays dividends at the European Indoors which take place on 5-6 March."I am quite confident," said Claxton.'

In [49]:
from rouge_metric import PyRouge


hypotheses = list()
# Pre-process and tokenize the summaries as you like
hypotheses.append(final_sentences.split())


# hypotheses

In [50]:

references = []
# Pre-process and tokenize the summaries as you like
references.append(actual_sentences.split())

# references

In [51]:

#Evaluate on tokenized documents
rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
scores = rouge.evaluate_tokenized(hypotheses, references)
print(scores)


{'rouge-1': {'r': 0.9935760171306209, 'p': 0.010096833859210097, 'f': 0.019990521735384088}, 'rouge-2': {'r': 0.9388297872340425, 'p': 0.007696668410954125, 'f': 0.015268166089965396}, 'rouge-4': {'r': 0.7259615384615384, 'p': 0.0033054594807582854, 'f': 0.006580954456308563}, 'rouge-l': {'r': 0.9935760171306209, 'p': 0.010096833859210097, 'f': 0.019990521735384088}, 'rouge-w-1.2': {'r': 0.7352461428984891, 'p': 0.00769613089289468, 'f': 0.015232813514207497}, 'rouge-s4': {'r': 0.960352422907489, 'p': 0.0047721203099689155, 'f': 0.009497048509006949}, 'rouge-su4': {'r': 0.9688947716743879, 'p': 0.005337727965465192, 'f': 0.010616966114908352}}
